In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import pandas as pd
try:
    dataset = pd.read_csv('/content/Churn_Modelling.csv')
    original_df=dataset.copy()
except FileNotFoundError:
    print("Error: The file 'Churn_Modelling.csv' was not found.")
    exit()


dataset.head()

dataset.shape

dataset.dtypes

# Drop irrelevant columns as they do not provide value for the model
dataset.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
print("Remaining columns:\n")
dataset.columns.to_list()

obj_columns=['Gender','Geography']
for column in obj_columns:
    dataset[column] = dataset[column].astype('category')

dataset.dtypes

dataset.isnull().sum()

for col in dataset.columns:
    print(col," : ",dataset[col].unique())
    print()

import matplotlib.pyplot as plt
dataset.hist(figsize=(12,8))
plt.show()

import seaborn as sns
#Count Plots for Categorical Features
for col in ['Gender', 'Geography', 'HasCrCard', 'IsActiveMember', 'NumOfProducts']:
    sns.countplot(x=col, data=dataset)
    plt.show()

print("Churn Rate:\n", dataset['Exited'].value_counts(normalize=True))
sns.countplot(x='Exited', data=dataset)
plt.title("Churn Distribution")
plt.show()

# Churn vs Gender
sns.countplot(x='Gender', hue='Exited', data=dataset)
plt.title("Churn by Gender")
plt.show()

# Churn vs Geography
sns.countplot(x='Geography', hue='Exited', data=dataset)
plt.title("Churn by Geography")
plt.show()

# Correlation Heatmap
corr = dataset.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Boxplots of Numerical vs Churn
for col in ['Age','Balance','CreditScore','EstimatedSalary']:
    sns.boxplot(x='Exited', y=col, data=dataset)
    plt.title(f"{col} vs Churn")
    plt.show()

# Separate features and target
X = dataset.drop('Exited', axis=1)
y = dataset['Exited']

# Identify categorical and numerical columns
categorical_cols = ['Gender', 'Geography']
numeric_cols = [col for col in X.columns if col not in categorical_cols]


# One-Hot Encode categorical features (drop first to avoid dummy variable trap)
X = pd.get_dummies(X, columns=['Gender', 'Geography'], drop_first=True)


print("After encoding:")
X.head()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


X['Gender_Male'] = X['Gender_Male'].astype(int)
X['Geography_Germany'] = X['Geography_Germany'].astype(int)
X['Geography_Spain'] = X['Geography_Spain'].astype(int)


print("After scaling:")
X.head()

from sklearn.model_selection import train_test_split
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

y_pred_log = log_model.predict(X_test)
print("Logistic Regression Accuracy:")
print(accuracy_score(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log))

rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
print("\nRandom Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

xgb_model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
print("\nXGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))

# Make Predictions & Probabilities
y_pred_log = log_model.predict(X_test)
y_prob_log = log_model.predict_proba(X_test)[:,1]

y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:,1]

y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:,1]

#Add CustomerId from original_df (align with X_test indices)
customer_ids = original_df.loc[X_test.index, "CustomerId"]

#Export Predictions + Probabilities with CustomerId
results = pd.DataFrame({
    "CustomerId": customer_ids,
    "Actual": y_test,
    "LogReg_Pred": y_pred_log,
    "LogReg_Prob": y_prob_log,
    "RF_Pred": y_pred_rf,
    "RF_Prob": y_prob_rf,
    "XGB_Pred": y_pred_xgb,
    "XGB_Prob": y_prob_xgb
})

results.to_csv("churn_predictions.csv", index=False)

# Logistic Regression
log_importances = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": log_model.coef_[0]
}).sort_values(by="Importance", key=abs, ascending=False)
log_importances.to_csv("log_reg_feature_importance.csv", index=False)

# Random Forest
rf_importances = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": rf_model.feature_importances_
}).sort_values(by="Importance", ascending=False)
rf_importances.to_csv("rf_feature_importance.csv", index=False)

# XGBoost
xgb_importances = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": xgb_model.feature_importances_
}).sort_values(by="Importance", ascending=False)
xgb_importances.to_csv("xgb_feature_importance.csv", index=False)